In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder as OHE
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
from tensorflow.keras import models
from tensorflow.keras import layers

In [6]:
df = pd.read_csv('FINAL_data2.csv')

In [7]:
df

,text,dialect,word_count
0,المجتمع مثلكم بتقول عيونك زايغه تطالع النسوان ...,Khaleeji,14
1,لكل فعل رده فعل المصرين بداو بالسب والشتم وسائ...,Khaleeji,14
2,تحسين بضمير تري بنتك تقدر شي عكس حتنقهر عشانك ...,Khaleeji,14
3,مجانين لولوه الاكثر مصيبه الفئه الي تدافع تعز ...,Khaleeji,14
4,بمقوله غيره ارتاي الحق دربه فهو لحق اسبق يصرفه...,Khaleeji,14
...,...,...,...
125916,حتي اولاد الرقاصه بقو عندهم خشم يتكلمو الجنيه ...,Sudani,13
125917,اختلاف كبير الشبكات الثلاث فحين تكون احداها جي...,Sudani,13
125918,دراسه منكم عشان تقدموا خدمه عملاء زي العالم وا...,Sudani,13
125919,اظن اغنيه بتقول اصلو الفرج عمرو قصير واظن بغني...,Sudani,13


In [10]:
dialect = pd.get_dummies(df['dialect'])

In [12]:
dialect = dialect.values

In [13]:
features = df.text.values.astype(str)

In [16]:
vocab_size = 1500000
tokenizer = Tokenizer(num_words=vocab_size, lower=False)
tokenizer.fit_on_texts(features)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    features, dialect, random_state=42, test_size=0.1, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, random_state=42, test_size=0.1, shuffle=True)

In [18]:
X_train_tok = tokenizer.texts_to_sequences(X_train)
X_train_tok = pad_sequences(X_train_tok, maxlen=20)

X_val_tok = tokenizer.texts_to_sequences(X_val)
X_val_tok = pad_sequences(X_val_tok, maxlen=20)

X_test_tok = tokenizer.texts_to_sequences(X_test)
X_test_tok = pad_sequences(X_test_tok, maxlen=20)

In [ ]:
model = models.Sequential()
model.add(Embedding(vocab_size, dialect.shape[1], input_shape=(20,)))
model.add(SpatialDropout1D(0.5))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy',
            optimizer='adam', metrics=['accuracy'])

In [27]:
checkpoint = ModelCheckpoint("lstm/lstm_best_model.h5", monitor='loss',
                            verbose=1, save_best_only=True, mode='auto', period=1)

In [ ]:
# we stoped the model as val_loss was not improving much
history = model.fit(X_train_tok, y_train, epochs=10, batch_size=15,
                    validation_data=(X_val_tok, y_val), callbacks=[checkpoint])

Epoch 1/10
6800/6800 [==============================] - ETA: 0s - loss: 1.5024 - accuracy: 0.3984
Epoch 1: loss improved from inf to 1.50244, saving model to lstm\lstm_best_model.h5
6800/6800 [==============================] - 643s 94ms/step - loss: 1.5024 - accuracy: 0.3984 - val_loss: 1.1175 - val_accuracy: 0.6058
Epoch 2/10
6800/6800 [==============================] - ETA: 0s - loss: 1.1735 - accuracy: 0.5485
Epoch 2: loss improved from 1.50244 to 1.17352, saving model to lstm\lstm_best_model.h5
6800/6800 [==============================] - 806s 118ms/step - loss: 1.1735 - accuracy: 0.5485 - val_loss: 0.9860 - val_accuracy: 0.6426
Epoch 3/10
6800/6800 [==============================] - ETA: 0s - loss: 1.0259 - accuracy: 0.6090
Epoch 3: loss improved from 1.17352 to 1.02585, saving model to lstm\lstm_best_model.h5
6800/6800 [==============================] - 1005s 148ms/step - loss: 1.0259 - accuracy: 0.6090 - val_loss: 0.9529 - val_accuracy: 0.6557
Epoch 4/10
6800/6800 [=============

KeyboardInterrupt: 

# Model analysis

In [29]:
model = models.load_model("lstm/lstm_best_model.h5")

In [ ]:
test_loss, test_acc = model.evaluate(X_test_tok, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

394/394 - 4s - loss: 1.0050 - accuracy: 0.6682 - 4s/epoch - 9ms/step

Test accuracy: 0.6682283878326416
